In [ ]:
# Cell 1: Install Libraries
!pip install numpy pandas scikit-learn matplotlib

In [ ]:
#  Cell 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Cell 3: Neural Network Class
class NeuralNetwork:
    """
    A class representing a simple Artificial Neural Network for predicting concrete compressive strength.
    """
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weights_ih = np.random.uniform(-1, 1, (hidden_size, input_size))
        self.bias_h = np.random.uniform(-1, 1, hidden_size)
        self.weights_ho = np.random.uniform(-1, 1, hidden_size)
        self.bias_o = np.random.uniform(-1, 1)
        self.alpha = 1  # shape parameter for sigmoid
        self.learning_rate = 0.02 # learning rate parameter
        self.training_errors = []

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-self.alpha * x))

    def forward(self, inputs):
        hidden_input = np.dot(self.weights_ih , inputs) + self.bias_h
        hidden_output = self.sigmoid(hidden_input)
        output_layer_output =  np.dot(self.weights_ho , hidden_output ) + self.bias_o
        return hidden_output, output_layer_output

    def backward(self, inputs,  desired_output, hidden_output,  output_layer_output):
        output_error = desired_output - output_layer_output
        hidden_error =   output_error *  (hidden_output * (1 - hidden_output)  * self.weights_ho)
        dw_output  =   self.learning_rate  * hidden_output * output_error
        db_output =    self.learning_rate  *   output_error
        dw_hidden =   np.outer (self.learning_rate * hidden_error ,inputs )
        db_hidden =   self.learning_rate * hidden_error
        self.bias_o  +=  db_output
        self.weights_ho  +=  dw_output
        self.weights_ih +=   dw_hidden
        self.bias_h  += db_hidden

    def train(self, training_data, desired_outputs,  iterations=12040):
        for i in range (iterations):
            total_error= 0
            for inputs , desired_output  in zip (training_data, desired_outputs):
                hidden_output, output_layer_output  = self.forward(inputs)
                self.backward(inputs, desired_output ,hidden_output,  output_layer_output)
                total_error += (desired_output - output_layer_output) ** 2

            avg_error = np.mean(total_error)
            self.training_errors.append(avg_error)
            if i % 1000 == 0:
                print(f"Iteration {i}, Error: {avg_error:.4f}")

    def predict(self, inputs):
        predictions = []
        for input_sample in inputs:
            _, output_layer_output  = self.forward(input_sample)
            predictions.append(output_layer_output)
        return np.array(predictions)

In [ ]:
# Cell 4: Plotting Functions
def plot_loss_curve(training_errors):
    plt.figure(figsize=(10, 6))
    plt.plot(training_errors)
    plt.title('Training Loss Curve')
    plt.xlabel('Iteration')
    plt.ylabel('Mean Squared Error')
    plt.grid(True)
    plt.show()

def plot_actual_vs_predicted(y_actual, y_predicted, title):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_actual, y_predicted)
    plt.plot([min(y_actual), max(y_actual)], [min(y_actual), max(y_actual)], 'k--', lw=2)  # Diagonal line
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(title)
    plt.grid(True)
    plt.show()

def plot_residuals(y_actual, y_predicted, title):
    residuals = y_actual - y_predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_predicted, residuals)
    plt.axhline(y=0, color='k', linestyle='--', lw=2)
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals (Actual - Predicted)')
    plt.title(title)
    plt.grid(True)
    plt.show()

In [ ]:
# Cell 5: Data Loading and Preprocessing
excel_file = 'your_dataset.xlsx'
df = pd.read_excel(excel_file)

X = df[['W/C RATIO', 'MIX PRAPOTIONS', 'CEMENT CONTENT(kg/m^3)', 'FLY ASH CONTENT(FA)kg/m^3']].copy()
y = df['28D(mpa)'].values

def extract_proportions(mix_str):
    try:
        parts = mix_str.split(':')
        return list(map(float, parts[1:]))
    except:
        return [np.nan, np.nan, np.nan, np.nan]  # Return 4 NaNs

proportions = X['MIX PRAPOTIONS'].apply(extract_proportions)
X[['Cement_Prop', 'FA_Prop', 'Sand_Prop', 'CA_Prop']] = pd.DataFrame(proportions.tolist(), index=X.index)
X.drop('MIX PRAPOTIONS', axis=1, inplace=True)
X.fillna(X.mean(), inplace=True)

X = X.values
y = y.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

input_scaler = MinMaxScaler()
X_train_scaled = input_scaler.fit_transform(X_train)
X_test_scaled = input_scaler.transform(X_test)

output_scaler = MinMaxScaler()
y_train_scaled = output_scaler.fit_transform(y_train)
y_test_scaled = output_scaler.transform(y_test)

In [ ]:
# Cell 6: Model Initialization and Training
input_size = X_train_scaled.shape[1]
hidden_size = 7
nn = NeuralNetwork(input_size, hidden_size)
nn.train(X_train_scaled, y_train_scaled.flatten())

In [ ]:
# Cell 7: Model Evaluation
y_train_pred_scaled = nn.predict(X_train_scaled)
y_test_pred_scaled = nn.predict(X_test_scaled)

y_train_pred = output_scaler.inverse_transform(y_train_pred_scaled.reshape(-1, 1))
y_test_pred = output_scaler.inverse_transform(y_test_pred_scaled.reshape(-1, 1))

# Calculate Performance Metrics
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("--- Training Performance ---")
print(f"Train Mean Squared Error: {train_mse:.2f}")
print(f"Train Root Mean Squared Error: {train_rmse:.2f}")
print(f"Train R-squared: {train_r2:.4f}")

print("\n--- Testing Performance ---")
print(f"Test Mean Squared Error: {test_mse:.2f}")
print(f"Test Root Mean Squared Error: {test_rmse:.2f}")
print(f"Test R-squared: {test_r2:.4f}")

In [ ]:
# Cell 8: Plotting Results
plot_loss_curve(nn.training_errors)
plot_actual_vs_predicted(y_train, y_train_pred, 'Actual vs. Predicted values (Training Set)')
plot_residuals(y_train, y_train_pred, 'Residual Plot (Training Set)')
plot_actual_vs_predicted(y_test, y_test_pred, 'Actual vs. Predicted values (Testing Set)')
plot_residuals(y_test, y_test_pred, 'Residual Plot (Testing Set)')

In [ ]:
# Cell 9: Prediction on New Data
new_data = np.array([[0.48, 400, 0, 1.15, 1.30, 2.70, 3.05]])
new_data_df = pd.DataFrame(new_data, columns=['W/C RATIO', 'CEMENT CONTENT(kg/m^3)', 'FLY ASH CONTENT(FA)kg/m^3', 'Cement_Prop', 'FA_Prop', 'Sand_Prop', 'CA_Prop'])
new_data_scaled = input_scaler.transform(new_data_df)
predicted_strength_scaled = nn.predict(new_data_scaled)
predicted_strength = output_scaler.inverse_transform(predicted_strength_scaled.reshape(-1, 1))
print("\nPredicted compressive strength for new data:", predicted_strength[0][0])